In [81]:
import torchvision
import torch
from torchvision import models
import torch.nn as nn
import torchvision.transforms as trans
import zipfile
import os
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import random
import numpy as np
import copy

In [82]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)  # Needed for multi-GPU

torch.backends.cudnn.deterministic = True  # Slower, but deterministic
torch.backends.cudnn.benchmark = False     # Avoids non-deterministic CuDNN paths


In [83]:
transformer = {
    'Train': trans.Compose([
        trans.RandomResizedCrop(224),
        trans.RandomHorizontalFlip(),
        trans.ToTensor(),
        trans.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'Val' : trans.Compose ([
        trans.Resize(256),
        trans.CenterCrop(224),
        trans.ToTensor(),
        trans.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
  }

In [84]:
DATA_DIR = 'input_data/all_data'

In [85]:
with zipfile.ZipFile('input_data/Classification+Dataset.zip', 'r') as zf:
  zf.extractall(DATA_DIR)

In [86]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DATA_DIR += '/Classification Dataset'

In [87]:
datasets = {x : torchvision.datasets.ImageFolder(os.path.join(DATA_DIR, x), transformer[x]) for x in transformer.keys()}
data_loaders = {x: torch.utils.data.DataLoader(datasets[x], batch_size=5, shuffle=True, num_workers=0) for x in transformer.keys()}
class_names = datasets['Val'].classes
dataset_sizes = {x: len(datasets[x]) for x in ['Train', 'Val']}

In [88]:
resnet101 = models.resnet101(weights=True)

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [89]:
for param in resnet101.parameters():
  param.requires_grad = True

In [90]:
in_features = resnet101.fc.in_features
resnet101.fc = nn.Linear(in_features, len(class_names))

In [91]:
resnet101_ft = resnet101.to(device)

In [92]:
optimizer = optim.Adam(resnet101.parameters(), lr=0.00001)
decay_lr = lr_scheduler.StepLR(optimizer, gamma=0.1, step_size=7)
loss_function = nn.CrossEntropyLoss()

In [93]:
def train_model(model, lossFunction, optimizer, scheduler, num_epochs=100):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['Train', 'Val']:
            if phase == 'Train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in data_loaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'Train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = lossFunction(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'Train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'Train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'Val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [94]:
train_model(
    resnet101_ft,
    loss_function,
    optimizer,
    decay_lr,
    20
)

Epoch 1/20
----------
Train Loss: 0.9056 Acc: 0.6399
Val Loss: 0.5016 Acc: 0.9400

Epoch 2/20
----------
Train Loss: 0.6606 Acc: 0.7653
Val Loss: 0.3264 Acc: 0.9600

Epoch 3/20
----------
Train Loss: 0.5217 Acc: 0.8264
Val Loss: 0.1796 Acc: 1.0000

Epoch 4/20
----------
Train Loss: 0.4741 Acc: 0.8392
Val Loss: 0.4828 Acc: 0.9400

Epoch 5/20
----------
Train Loss: 0.4091 Acc: 0.8360
Val Loss: 0.1736 Acc: 0.9800

Epoch 6/20
----------
Train Loss: 0.4040 Acc: 0.8682
Val Loss: 0.1313 Acc: 1.0000

Epoch 7/20
----------
Train Loss: 0.3442 Acc: 0.8682
Val Loss: 0.1114 Acc: 0.9800

Epoch 8/20
----------
Train Loss: 0.3097 Acc: 0.9068
Val Loss: 0.1003 Acc: 0.9800

Epoch 9/20
----------
Train Loss: 0.2839 Acc: 0.9100
Val Loss: 0.3933 Acc: 0.9800

Epoch 10/20
----------
Train Loss: 0.3359 Acc: 0.8746
Val Loss: 0.1078 Acc: 1.0000

Epoch 11/20
----------
Train Loss: 0.2427 Acc: 0.9068
Val Loss: 0.2005 Acc: 0.9800

Epoch 12/20
----------
Train Loss: 0.3158 Acc: 0.8875
Val Loss: 0.3458 Acc: 0.9800

E

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 